# Modding Minecraft
## Modding Python

## Agenda

https://github.com/tobynance/okcpython.git

Found in **`presentations/2015_07_08_modding_minecraft`**

* Getting Up and Running with CanaryMod
* Floor of Dirt
* Personal Bridge
* Skating Rink
* Build a House


I want to thank Cody Piersall for introducting me to ipython/jupyter, and Aaron Krauss for providing an excellent demonstration of its use at last month's Ruby meetup.

## Installation
<div style="float: right">
  ![Adventures In Minecraft](AdventuresInMinecraftCover.jpg)
</div>

Go to http://www.wiley.com/WileyCDA/Section/id-823690.html and download the **Starter Kit** for your platform (I used the Mac Starter Kit to run on Linux).  The **readme.txt** included will walk you through getting up and running.  It looks like **Bukkit** is currently battling a DMCA takedown, so I used **CanaryMod**.

The major components you will need:

* Minecraft (https://minecraft.net)
* CanaryMod (http://canarymod.net/releases)
* RaspberryJuice (https://github.com/martinohanlon/canaryraspberryjuice)

There is a curious version of Minecraft written in Python for the Raspberry Pi.

The API for it has been ported to the primary version of Minecraft in a project called **Raspberry Juice**.  We will be using this library, along with a custom server called **CanaryMod** for this presentation.

![Not Found](not_found.png)

## Installation
<div style="float: right">
  ![Adventures In Minecraft](AdventuresInMinecraftCover.jpg)
</div>

Go to http://www.wiley.com/WileyCDA/Section/id-823690.html and download the **Starter Kit** for your platform (I used the Mac Starter Kit to run on Linux).  The **readme.txt** included will walk you through getting up and running.  It looks like **Bukkit** is currently battling a DMCA takedown, so I used **CanaryMod**.

The major components you will need:

* Minecraft (https://minecraft.net)
* <s>CanaryMod (http://canarymod.net/releases)</s> **(Down as of last night)**
* RaspberryJuice (https://github.com/martinohanlon/canaryraspberryjuice)


## Installation

* Minecraft (https://minecraft.net)
* Download starter kit: http://www.wiley.com/WileyCDA/Section/id-823690.html

<pre>

cd Bukkit
./start_server.command
defaultgamemode creative
op [username]

<pre>

### Connect to your local <s>CanaryMod</s> <span style="color: blue; font-weight: bold">Bukkit</span> server
  + In Minecraft, choose **Multiplayer**, click Add server
  + For the **Server Name** I used *`Canary`*
  + For the **Server Address** put *`localhost`*
  + Click **Done**
  
![Add Server](minecraft_adding_server.png)
  

You should now have the Canary server showing in the Server List

![Server List](minecraft_server_list.png)

## Finally, some code

### Floor of Dirt (00_floor_of_dirt.py)

In [ ]:
from mcpi import block
from mcpi import minecraft

########################################################################
def floor_of_dirt():
    world = minecraft.Minecraft.create()
    player = world.player

    pos = player.getTilePos()
    width = 10
    world.setBlocks(pos.x-width,
                    pos.y-1,
                    pos.z-width,
                    pos.x+width,
                    pos.y-1,
                    pos.z+width,
                    block.DIRT.id)

########################################################################
floor_of_dirt()

#### Floor of Dirt continued (01_floor_of_dirt.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block

########################################################################
class Mod(BaseCommand):
    ####################################################################
    def once(self):
        pos = self.player.getTilePos()
        width = 10
        self.world.setBlocks(pos.x-width,
                             pos.y-1,
                             pos.z-width,
                             pos.x+width,
                             pos.y-1,
                             pos.z+width,
                             block.DIRT.id)

########################################################################
if __name__ == "__main__":
    Mod().once()

### Safe Walk (02_safe_walk.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block

########################################################################
class Mod(BaseCommand):
    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        pos.y -= 1  # block BELOW the player
        self.set_block(pos, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()

#### Safe Walk continued (03_safe_walk_platform.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]


########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.01

    ####################################################################
    def get_surrounding_blocks(self, pos, distance=1):
        """
        Get all the positions around `pos` on the XZ plane,
        including the `pos` block.
        """
        for x in range(-distance, distance+1):
            for z in range(-distance, distance+1):
                yield Vec3(pos.x+x, pos.y, pos.z+z)

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        pos.y -= 1  # block BELOW the player

        for p in self.get_surrounding_blocks(pos):
            b = self.get_block(p)
            if b in REPLACE_BLOCKS:
                self.set_block(p, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()

#### Safe Walk continued (04_safe_walk_faster_platform.py and base_command.py)

In [ ]:
# in base_command.BaseCommand

    ####################################################################
    def get_blocks(self, start_pos, end_pos):
        args = list(start_pos) + list(end_pos)
        blocks = self.world.getBlocks(*args)
        block_index = 0
        for x in range(start_pos.x, end_pos.x+1):
            for y in range(start_pos.y, end_pos.y+1):
                for z in range(start_pos.z, end_pos.z+1):
                    yield (Vec3(x, y, z), blocks[block_index])
                    block_index += 1

In [ ]:
# 04_safe_walk_faster_platform.py
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]


########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.05

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        blocks = self.get_blocks(Vec3(pos.x-1,
                                      pos.y-2,
                                      pos.z-1),
                                 Vec3(pos.x+1,
                                      pos.y-1,
                                      pos.z+1))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.set_block(p, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()


#### Safe Walk continued (05_safe_walk_faster_platform.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]

########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.05

    ####################################################################
    def __init__(self):
        super(Mod, self).__init__()
        self.old_ice_blocks = set()

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        for p, b in self.old_ice_blocks.copy():
            if p.y >= pos.y or (pos - p).length() > 4:
                self.set_block(p, b)
                self.old_ice_blocks.remove((p, b))

        blocks = self.get_blocks(Vec3(pos.x-2,
                                      pos.y-2,
                                      pos.z-2),
                                 Vec3(pos.x+2,
                                      pos.y-1,
                                      pos.z+2))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.old_ice_blocks.add((p, b))
                self.set_block(p, block.ICE.id)


########################################################################
if __name__ == "__main__":
    Mod().run()

### Skating Rink (06_skating_rink.py)

In [ ]:
import time
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.ICE.id]

RANGE = 5

########################################################################
class Mod(BaseCommand):
    ####################################################################
    def once(self):
        self.world.postToChat("Sleeping...")
        time.sleep(4)
        self.world.postToChat("Freeze!")
        self.replaced = set()
        self.to_replace = set()
        pos = self.player.getTilePos()

        blocks = self.get_blocks(Vec3(pos.x-RANGE,
                                      pos.y-1,
                                      pos.z-RANGE),
                                 Vec3(pos.x+RANGE,
                                      pos.y-1,
                                      pos.z+RANGE))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.to_replace.add(p)
        self.make_rink()
        self.world.postToChat("All Done.")

    ####################################################################
    def make_rink(self):
        while len(self.to_replace) > 0:
            pos = self.to_replace.pop()
            print "replacing block at", pos
            self.set_block(pos, block.ICE.id)
            self.replaced.add(pos)

            # check all neighbors
            for x in [-1, 0, 1]:
                for z in [-1, 0, 1]:
                    if x == z == 0:
                        continue
                    new_pos = Vec3(pos.x+x,
                                   pos.y,
                                   pos.z+z)
                    if new_pos in self.replaced or new_pos in self.to_replace:
                        continue
                    b = self.get_block(new_pos)
                    if b in REPLACE_BLOCKS:
                        self.to_replace.add(new_pos)

########################################################################
if __name__ == "__main__":
    Mod().once()

#### Further Development

* Fetch chunks of blocks at once and cache them
* Cache writes of multiple blocks

### Build (07_build.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block


########################################################################
class Mod(BaseCommand):
    HOUSE_WIDTH = 12
    HOUSE_HEIGHT = 4

    ####################################################################
    def buildHouse(self, x, y, z):
        self.clearHouse(x, y, z)

        # draw floor
        self.world.setBlocks(x,y-1,z,x+self.HOUSE_WIDTH,y-1,z+self.HOUSE_WIDTH,block.GRASS.id)
    
        # draw walls
        self.world.setBlocks(x, y, z, x+self.HOUSE_WIDTH, y+self.HOUSE_HEIGHT, z, block.STONE.id)
        self.world.setBlocks(x+self.HOUSE_WIDTH, y, z, x+self.HOUSE_WIDTH, y+self.HOUSE_HEIGHT, z+self.HOUSE_WIDTH, block.STONE.id)
        self.world.setBlocks(x+self.HOUSE_WIDTH, y, z+self.HOUSE_WIDTH, x, y+self.HOUSE_HEIGHT, z+self.HOUSE_WIDTH, block.STONE.id)
        self.world.setBlocks(x, y, z+self.HOUSE_WIDTH, x, y+self.HOUSE_HEIGHT, z, block.STONE.id)
    
        # draw windows
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)-2,y+1,z,x+(self.HOUSE_WIDTH/2)-2,y+2,z,block.GLASS.id)
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)+2,y+1,z,x+(self.HOUSE_WIDTH/2)+2,y+2,z,block.GLASS.id)

        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)-2,y+1,z+self.HOUSE_WIDTH,x+(self.HOUSE_WIDTH/2)+2,y+2,z+self.HOUSE_WIDTH,block.GLASS.id)
        self.world.setBlocks(x,y+1,z+(self.HOUSE_WIDTH/2)-2,x,y+2,z+(self.HOUSE_WIDTH/2)+2,block.GLASS.id)
        self.world.setBlocks(x+self.HOUSE_WIDTH,y+1,z+(self.HOUSE_WIDTH/2)-2,x+self.HOUSE_WIDTH,y+2,z+(self.HOUSE_WIDTH/2)+2,block.GLASS.id)

        # cobble arch
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)-1,y,z,x+(self.HOUSE_WIDTH/2)+1,y+2,z,block.COBBLESTONE.id)
        # clear space for door
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2),y,z,x+(self.HOUSE_WIDTH/2),y+1,z,block.AIR.id)

        # small porch
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)-2,y-1,z,x+(self.HOUSE_WIDTH/2)+2,y-1,z-2,block.WOOD_PLANKS.id)
        self.world.setBlocks(x+(self.HOUSE_WIDTH/2)-2,y,z-1,x+(self.HOUSE_WIDTH/2)+2,y+3,z-2,block.AIR.id)

        # torches weren't working, so add some glow stones for light
        self.world.setBlock(x+(self.HOUSE_WIDTH/2)-1,y+3,z,block.GLOWSTONE_BLOCK.id)
        self.world.setBlock(x+(self.HOUSE_WIDTH/2)+1,y+3,z,block.GLOWSTONE_BLOCK.id)

        self.world.setBlock(x+(self.HOUSE_WIDTH/2)-1,y+3,z+self.HOUSE_WIDTH,block.GLOWSTONE_BLOCK.id)
        self.world.setBlock(x+(self.HOUSE_WIDTH/2)+1,y+3,z+self.HOUSE_WIDTH,block.GLOWSTONE_BLOCK.id)

        self.world.setBlock(x,y+3,z+(self.HOUSE_WIDTH/2)-1,block.GLOWSTONE_BLOCK.id)
        self.world.setBlock(x,y+3,z+(self.HOUSE_WIDTH/2)+1,block.GLOWSTONE_BLOCK.id)

        self.world.setBlock(x+self.HOUSE_WIDTH,y+3,z+(self.HOUSE_WIDTH/2)-1,block.GLOWSTONE_BLOCK.id)
        self.world.setBlock(x+self.HOUSE_WIDTH,y+3,z+(self.HOUSE_WIDTH/2)+1,block.GLOWSTONE_BLOCK.id)

        # draw roof
        self.world.setBlocks(x,y+self.HOUSE_HEIGHT+1,z,x+self.HOUSE_WIDTH,y+self.HOUSE_HEIGHT+1,z+self.HOUSE_WIDTH,block.WOOD_PLANKS.id)

        # Add a woolen carpet, the colour is 14, which is red.
        self.world.setBlocks(x+1, y-1, z+1, x+self.HOUSE_WIDTH-1, y-1, z+self.HOUSE_WIDTH-1, block.WOOL.id, 14)

        # draw door
        self.world.setBlock(x+(self.HOUSE_WIDTH/2),y,z,block.DOOR_WOOD.id, 1)
        self.world.setBlock(x+(self.HOUSE_WIDTH/2),y+1,z,block.DOOR_WOOD.id, 8)

        # draw torches
        self.world.setBlock(x+(self.HOUSE_WIDTH/2)-1,y+2,z-1,block.TORCH.id,4)
        self.world.setBlock(x+(self.HOUSE_WIDTH/2)+1,y+2,z-1,block.TORCH.id,4)

    ####################################################################
    def clearHouse(self, x, y, z):
        self.world.setBlocks(x-2,y-1,z-2,x+self.HOUSE_WIDTH+2,y+self.HOUSE_HEIGHT+1,z+self.HOUSE_WIDTH+2,block.AIR.id)

    ####################################################################
    def once(self):
        pos = self.player.getTilePos()
        self.buildHouse(pos.x+2, pos.y, pos.z)

########################################################################
if __name__ == "__main__":
    Mod().once()


#### Further Development

* Make the building take into account the orientation of the player
* Make a Village